In [1]:
##########
# basics #
##########

import warnings
warnings.filterwarnings('ignore')
import datetime
import glob
import hashlib
import itertools
import math
import operator
import os
import pickle
import random
import re
import string
import sys
import time

###########
# science #
###########

import scipy as sp
import numpy as np
import pandas as pd
rseed = random.seed(42)

######
# ml #
######

import xgboost as xgb
import theano as thno
from sklearn import decomposition
from sklearn import preprocessing
from sklearn import metrics
from sklearn import cross_validation
from sklearn import grid_search
from sklearn import pipeline
from sklearn import feature_selection

#################
# visualization #
#################

# plotly
import plotly.plotly as py
import plotly.tools as tls
from plotly.graph_objs import *
import cufflinks as cf
tls.set_credentials_file(username=os.environ.get('PLOTLY_USERNAME'), api_key=os.environ.get('PLOTLY_APIKEY'))
cf.set_config_file(offline=False, world_readable=True, theme='pearl')

# matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'svg')
mpl.rcParams['figure.figsize']=(12.0,4.0)
%matplotlib inline

# seaborn
import seaborn as sns
sns.set_style('darkgrid')
sns.set_palette('muted', n_colors=15, desat=None)
sns.set_context("notebook", font_scale=1.5,
                rc={"lines.linewidth": 2.5})

#graphistry
import graphistry
graphistry.register(key=os.environ.get('GRAPHISTRY_APIKEY'))

############
# sys info #
############

%reload_ext watermark
%watermark -a "Ken Cavagnolo" -n -u -v -m -h -g -p numpy,scipy,pandas,\
scikit-learn,theano,xgboost,\
matplotlib,seaborn,plotly

A new version of PyGraphistry is available (installed=0.9.17 latest=0.9.30).
Ken Cavagnolo 
last updated: Sun Sep 11 2016 

CPython 2.7.10
IPython 4.1.1

numpy 1.11.1
scipy 0.18.0
pandas 0.18.1
scikit-learn 0.17.1
theano 0.8.2
xgboost 0.6a2
matplotlib 1.5.0
seaborn 0.8.dev0
plotly 1.9.0

compiler   : GCC 4.2.1 Compatible Apple LLVM 7.0.0 (clang-700.0.72)
system     : Darwin
release    : 15.6.0
machine    : x86_64
processor  : i386
CPU cores  : 4
interpreter: 64bit
host name  : DrGonzo.local
Git hash   : a32b6437f83e4bfe41624ea4b826e21f65815d76


# Vanilla

In [2]:
# sigmoid
def nonlin(x, deriv=False):
    if deriv==True:
        return x*(1-x)
    return 1/(1+np.exp(-x))

In [3]:
# data
X = np.array([[0,0,1],
              [0,1,1],
              [1,0,1],
              [1,1,1]])
y = np.array([[0,0,1,1]]).T

In [4]:
# random initial weights
syn0 = 2*np.random.random((3,1)) - 1

In [5]:
# train net
for iter in xrange(100):

    # forward propagation
    l0 = X
    l1 = nonlin(np.dot(l0,syn0))

    # how much did we miss?
    l1_err = y - l1

    # multiply how much we missed by the 
    # slope of the sigmoid at the values in l1
    l1_delta = l1_err * nonlin(l1,True)

    # update weights
    syn0 += np.dot(l0.T, l1_delta)
    print syn0

print "Output After Training:"
print l1

[[ 0.56666344]
 [-0.62705718]
 [ 0.13628696]]
[[ 0.7600985 ]
 [-0.59640514]
 [ 0.10740135]]
[[ 0.9279025 ]
 [-0.57977861]
 [ 0.05430837]]
[[ 1.07814177]
 [-0.57031076]
 [-0.01051622]]
[[ 1.21578491]
 [-0.56390798]
 [-0.07973971]]
[[ 1.3436899 ]
 [-0.55848675]
 [-0.14943214]]
[[ 1.46353943]
 [-0.55310749]
 [-0.21759968]]
[[ 1.57639617]
 [-0.54742002]
 [-0.28329733]]
[[ 1.68299765]
 [-0.54135498]
 [-0.34613461]]
[[ 1.78390682]
 [-0.53496335]
 [-0.4060088 ]]
[[ 1.87958736]
 [-0.52833737]
 [-0.46296325]]
[[ 1.97044117]
 [-0.52157412]
 [-0.51711259]]
[[ 2.05682709]
 [-0.51476031]
 [-0.56860353]]
[[ 2.13907057]
 [-0.50796746]
 [-0.61759413]]
[[ 2.21746904]
 [-0.50125166]
 [-0.66424301]]
[[ 2.29229515]
 [-0.49465522]
 [-0.70870382]]
[[ 2.36379916]
 [-0.48820891]
 [-0.75112243]]
[[ 2.43221082]
 [-0.48193415]
 [-0.79163573]]
[[ 2.49774106]
 [-0.47584503]
 [-0.83037129]]
[[ 2.56058349]
 [-0.4699499 ]
 [-0.86744745]]
[[ 2.62091587]
 [-0.46425274]
 [-0.90297366]]
[[ 2.67890144]
 [-0.45875426]
 [-0

# Keras

In [2]:
# load pima indians dataset
datadir = '/Users/cavagnolo/ml_fun/pima_indians/data/'
dataset = np.loadtxt(datadir+"pima-indians-diabetes.data.csv", delimiter=",")

# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

In [9]:
# create model
from keras.models import Sequential
from keras.layers import Dense, Activation
model = Sequential()
model.add(Dense(12, input_dim=8, init='uniform', activation='relu'))
model.add(Dense(8, init='uniform', activation='relu'))
model.add(Dense(1, init='uniform', activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(X, Y, nb_epoch=150, batch_size=10)

# evaluate the model
scores = model.evaluate(X, Y)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Epoch 1/150
768/768 [==============================] - 0s - loss: 0.6806 - acc: 0.6354     
Epoch 2/150
768/768 [==============================] - 0s - loss: 0.6637 - acc: 0.6510     
Epoch 3/150
768/768 [==============================] - 0s - loss: 0.6582 - acc: 0.6380     
Epoch 4/150
768/768 [==============================] - 0s - loss: 0.6481 - acc: 0.6510     
Epoch 5/150
768/768 [==============================] - 0s - loss: 0.6416 - acc: 0.6497     
Epoch 6/150
768/768 [==============================] - 0s - loss: 0.6304 - acc: 0.6693     
Epoch 7/150
768/768 [==============================] - 0s - loss: 0.6216 - acc: 0.6628     
Epoch 8/150
768/768 [==============================] - 0s - loss: 0.6150 - acc: 0.6823     
Epoch 9/150
768/768 [==============================] - 0s - loss: 0.6076 - acc: 0.6914     
Epoch 10/150
768/768 [==============================] - 0s - loss: 0.6050 - acc: 0.6836     
Epoch 11/150
768/768 [==============================] - 0s - loss: 0.5937 - acc